In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import re

model = LLM(
    "simplescaling/s1-32B",
    tensor_parallel_size=4,
)
tok = AutoTokenizer.from_pretrained("simplescaling/s1-32B")

stop_token_ids = tok("<|im_end|>")["input_ids"]

sampling_params = SamplingParams(
    max_tokens=32768,
    min_tokens=0,
    stop_token_ids=stop_token_ids,
)


config.json:   0%|          | 0.00/830 [00:00<?, ?B/s]

INFO 02-14 19:00:19 config.py:1861] Downcasting torch.float32 to torch.float16.
INFO 02-14 19:00:23 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 02-14 19:00:23 config.py:1020] Defaulting to use mp for distributed inference
INFO 02-14 19:00:23 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='simplescaling/s1-32B', speculative_config=None, tokenizer='simplescaling/s1-32B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConf

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

WARNING 02-14 19:00:24 multiproc_gpu_executor.py:56] Reducing Torch parallelism from 8 threads to 1 to avoid unnecessary CPU contention. Set OMP_NUM_THREADS in the external environment to tune this value as needed.
INFO 02-14 19:00:24 custom_cache_manager.py:17] Setting Triton cache manager to: vllm.triton_utils.custom_cache_manager:CustomCacheManager
INFO 02-14 19:00:24 selector.py:135] Using Flash Attention backend.
(VllmWorkerProcess pid=2869989) INFO 02-14 19:00:24 selector.py:135] Using Flash Attention backend.
(VllmWorkerProcess pid=2869989) INFO 02-14 19:00:24 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=2869990) INFO 02-14 19:00:24 selector.py:135] Using Flash Attention backend.
(VllmWorkerProcess pid=2869990) INFO 02-14 19:00:24 multiproc_worker_utils.py:215] Worker ready; awaiting tasks
(VllmWorkerProcess pid=2869988) INFO 02-14 19:00:24 selector.py:135] Using Flash Attention backend.
(VllmWorkerProcess pid=2869988) INFO 02-14 19:00:24 mu

model-00003-of-00029.safetensors:   0%|          | 0.00/4.47G [00:00<?, ?B/s]

...

model-00013-of-00029.safetensors:   0%|          | 0.00/4.47G [00:00<?, ?B/s]

model-00014-of-00029.safetensors:   0%|          | 0.00/4.72G [00:00<?, ?B/s]

model-00015-of-00029.safetensors:   0%|          | 0.00/4.47G [00:00<?, ?B/s]

model-00016-of-00029.safetensors:   0%|          | 0.00/4.47G [00:00<?, ?B/s]

model-00017-of-00029.safetensors:   0%|          | 0.00/4.72G [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/ssdscratch/byuan48/math_eval/data/AIME25.jsonl")
correct = 0
for i in range(len(dataset['train'])):
    prompt = dataset['train']['problem'][i]
    answer = dataset['train']['answer'][i]
    prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"
    o = model.generate(prompt, sampling_params=sampling_params)
    with open('output.log', 'w') as f:
        f.write(o[0].outputs[0].text)
    pattern = r"\\boxed\{(\d+)\}"
    matches = re.findall(pattern, o[0].outputs[0].text)
    print(f'matches: {matches}')
    if len(set(matches)) != 1:
        print(f'matches: {matches} not unique')
        print(f'current answer: {answer}')
    # remove the leading 0s
    matches[-1] = matches[-1].lstrip('0')
    if matches[-1] == str(answer):
        correct += 1
    print(f'correct: {correct}, in {i+1} questions')

print(f'correct: {correct}')



In [6]:
# right solution for the third question

prompt = "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n" + prompt + "<|im_end|>\n<|im_start|>assistant\n"

o = model.generate(prompt, sampling_params=sampling_params)
print(o[0].outputs[0].text)
pattern = r"\\boxed\{(\d+)\}"
matches = re.findall(pattern, o[0].outputs[0].text)
print(f'matches: {matches}')

Processed prompts: 100%|██████████| 1/1 [02:54<00:00, 174.09s/it, est. speed input: 0.81 toks/s, output: 33.67 toks/s]

think
Alright, let's tackle this problem. So, there are 9 baseball players, each getting a single scoop of ice cream in either chocolate, vanilla, or strawberry. The conditions are:

1. At least one player chose each flavor.
2. The number of chocolate is greater than vanilla, which in turn is greater than strawberry.

We need to find the number of different assignments of flavors to players that meet these conditions, then find the remainder when that number is divided by 1000.

Hmm, okay, first I need to figure out the possible distributions of the ice cream flavors. Since there are 9 players, and each player picks one of the three flavors, we need to partition the number 9 into three positive integers (since at least one player for each flavor) such that the numbers are in decreasing order: chocolate > vanilla > strawberry.

So we need to find all ordered triples (c, v, s) where:

- c + v + s = 9
- c > v > s
- c, v, s are positive integers.

And each of these triples represents a par